In [27]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import statsmodels
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
advertising_data = pd.read_csv('http://www-bcf.usc.edu/~gareth/ISL/Advertising.csv', index_col=0)
advertising_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 1 to 200
Data columns (total 4 columns):
TV           200 non-null float64
Radio        200 non-null float64
Newspaper    200 non-null float64
Sales        200 non-null float64
dtypes: float64(4)
memory usage: 7.8 KB


In [29]:
advertising_data.head()[['TV', 'Sales']]

,TV,Sales
1,230.1,22.1
2,44.5,10.4
3,17.2,9.3
4,151.5,18.5
5,180.8,12.9


In [30]:
# regression on how TV ad spending affects sales
import statsmodels.formula.api as sm
res = sm.ols("Sales ~ TV", advertising_data).fit()

In [31]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     312.1
Date:                Thu, 09 Feb 2017   Prob (F-statistic):           1.47e-42
Time:                        14:25:49   Log-Likelihood:                -519.05
No. Observations:                 200   AIC:                             1042.
Df Residuals:                     198   BIC:                             1049.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      7.0326      0.458     15.360      0.000         6.130     7.935
TV             0.0475      0.003     17.668      0.000         0.042     0.053
==============================================================================
Omnibus:                        0.531   Durbin-Watson:                   1.935
Prob(Omnibus):                  0.767   Jarque-Bera (JB):                0.669
Skew:                          -0.089   Prob(JB):                        0.716
Kurtosis:                       2.779   Cond. No.                         338.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
from statsmodels.stats.outliers_influence import summary_table
from statsmodels.sandbox.regression.predstd import wls_prediction_std

from statsmodels.stats.outliers_influence import summary_table
summ_table, data, ss2 = summary_table(res, alpha=0.05)


fittedvalues = data[:,2]
predict_mean_se  = data[:,3]
predict_mean_ci_low, predict_mean_ci_upp = data[:,4:6].T
predict_ci_low, predict_ci_upp = data[:,6:8].T

# wls cinv
prstd, iv_l, iv_u = wls_prediction_std(res)

In [33]:
summ_table

Obs,Dep Var,Predicted,Std Error,Mean ci,Mean ci,Predict ci,Predict ci,Residual,Std Error,Student,Cook's
,Population,Value,Mean Predict,95% low,95% upp,95% low,95% upp,,Residual,Residual,D
1.0,22.1,17.9707745128,0.3209911679,17.3377743194,18.6037747061,11.5135458897,24.4280031358,4.12922548723,3.24280835065,1.27334860427,0.00794343302167
2.0,10.4,9.1479740484,0.359466121714,8.43910057369,9.85684752311,2.68286663637,15.6130814604,1.2520259516,3.23876912365,0.386574622582,0.00092043396647
3.0,9.3,7.85022376458,0.418501179336,7.02493212443,8.67551540472,1.37131810566,14.3291294235,1.44977623542,3.23167109896,0.448615032604,0.00168755024544
4.0,18.5,14.2343945747,0.230733716317,13.7793836534,14.689405496,7.79217855969,20.6766105898,4.26560542527,3.2504773928,1.31230121296,0.00433875260405
5.0,12.9,15.6272181394,0.247677135601,15.1387944992,16.1156417797,9.18255601497,22.0718802639,-2.72721813942,3.24923027276,-0.83934283214,0.00204672562679
6.0,7.2,7.44616232089,0.437773799039,6.58286473785,8.30945990394,0.962305832448,13.9300188093,-0.246162320895,3.22911681886,-0.0762320890522,5.34044767361e-05
7.0,11.8,9.76595037403,0.333374313692,9.10853040076,10.4233703473,3.30628217302,16.225618575,2.03404962597,3.24155871394,0.627491218107,0.00208229277966
8.0,13.2,12.7464977292,0.241475286481,12.2703042438,13.2226912146,6.30275095564,19.1902445027,0.453502270823,3.24969706511,0.139552168014,5.37653904009e-05
9.0,4.8,7.44140865685,0.438002595821,6.57765988256,8.30515743115,0.957492078812,13.9253252349,-2.64140865685,3.22908579245,-0.818005103187,0.00615568278417


In [34]:
# mean squared error
se =np.sqrt(res.mse_resid)

3.2586563686504624

In [35]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std

pred = res.predict(exog={"TV":100})
_, pred_low, pred_high = wls_prediction_std(res, exog=[1,100], weights=1)



In [37]:
print(pred_low, pred_high)

[ 5.33925132] [ 18.23326387]


In [38]:
pred_low

array([ 5.33925132])

In [39]:
res.cov_params().shape

(2, 2)

In [40]:
st, data, ss2 = summary_table(res, alpha=0.05)

In [41]:
tv_bar = np.average(advertising_data['TV'])
tv_s = np.std(advertising_data['TV'])

In [42]:
tv_bar

147.04249999999999

In [43]:
from scipy import stats

x = 1 + 1/len(advertising_data) + (pow(100-tv_bar,2)/((len(advertising_data)-1)*pow(tv_s,2)))
conf_low = pred - stats.t.ppf(1-0.025, 198) * se*pow(x, 0.5)
conf_hi = pred + stats.t.ppf(1-0.025, 198) * se*pow(x, 0.5)

NameError: name 'se' is not defined

In [ ]:
print(conf_low, conf_hi)

In [ ]:
from scipy import stats

x = 1/len(advertising_data) + (pow(100-tv_bar,2)/((len(advertising_data)-1)*pow(tv_s,2)))
conf_low = pred - stats.t.ppf(1-0.025, 198) * se*pow(x, 0.5)
conf_hi = pred + stats.t.ppf(1-0.025, 198) * se*pow(x, 0.5)

In [ ]:
print(conf_low, conf_hi)

In [ ]:
data[:,4:6].T

In [ ]:
ci_low = data[:,4]
ci_hi = data[:,5]
pi_low = data[:,6]
pi_hi = data[:,7]

In [ ]:
# let's plot the data and fit
# source: http://mpastell.com/2013/04/19/python_regression/

plt.plot(advertising_data["TV"], advertising_data['Sales'], 'ro')
plt.plot(advertising_data["TV"], res.fittedvalues, 'b')
plt.plot(advertising_data["TV"], ci_low, 'g--')
plt.plot(advertising_data["TV"], ci_hi, 'g--')
plt.plot(advertising_data["TV"], pi_low, 'c--')
plt.plot(advertising_data["TV"], pi_hi, 'c--')



plt.legend(['TV data', 'Predicted values', 'Confidence Intervals', 'Prediction Intervals'])
plt.xlim(0, 320)
plt.xlabel('TV Ad Units (thousands)')
plt.ylabel('Sales Units (thousands)')
plt.title('Simple Linear Regression Sales ~ TV')

In [ ]:
# closer look at the residuals
# TODO confidence bounds - R style
plt.hist(res.wresid)
plt.ylabel('Count')
plt.xlabel('Residuals/std of Sales (Pearson)')

In [ ]:
# TODO clean up the extra cells generated
total = res
res.resid_pearson


In [ ]:
[x/rse for x in res.resid]

In [ ]:
# res.mse_total
res.resid_pearson

In [ ]:
res.ess

In [ ]:
res.uncentered_tss

In [ ]:
rss

### TODO note some impressions/assumptions we can make looking at this graph

- Looks like the variance in residuals is correlated to the predictive set (heteroskedasticity).



## Assessing the accuracy of the model

To do this we need to talk about the significance of the Residual standard error, R-squared, adjusted R squared and the F statistics

### RSE

The RSE is an estimate of the standard deviation of $\varepsilon$ residuals. Its formula:
$$
RSE = \sqrt{RSS/n-2}
$$

where RSS is the residual sum squares. What a good value for the RSS is dependent.

In [ ]:
# statsmodel lib didnt seem to provide this value
# hence I've calculated it manually

# rss = sum([pow(x - y, 2) for x,y in zip(advertising_data['Sales'], res.fittedvalues)])
# TODO fix this
rss = sum([pow(x - advertising_data["TV"], 2) for x in res.resid])
rse = pow(rss/(len(advertising_data['Sales'])-2), 0.5)

# residual standard error
rss

In [ ]:
sum([pow(x - advertising_data["TV"][ii], 2) for ii, x in enumerate(res.resid)])


In [ ]:
import math
math.sqrt(ans)
res.mse_total

In [ ]:
# few graphs on leverage
influence  = res.get_influence()
c,p = influence.cooks_distance
plt.stem(np.arange(len(c)), c, markerfmt=",")

# TODO talk a little about the significance of the graph below

In [ ]:
from statsmodels.graphics.regressionplots import plot_leverage_resid2, influence_plot
plot_leverage_resid2(res)
influence_plot(res)
# TODO talk about these incontext of outlier analysis
# TODO also figure out why dup graphs

A residual standard error value of 3.259 means the on average the actual sales will deviate from the (true?) regression line by 3.26 unitsm



### $ R^2 $

R-squared is a value between 0 and 1 which indicates what percentage of the variance is explained by the model. An R-squared value of 1 means that the model perfectly captures all the varience - including the irreducible errors (or that the irreducible error is 0), which is noise one usually should not be able to capture. It can also be thought of as the relationship between the predictor and the response.

### Adjusted $ R^2 $

Adjusted R-squared is the R squared statistic while controlling for the number of predictors in the model. While not relevant to simple linear regression, it is an important concept for multiple linear regression. Thus we will cover it on a later date

In [ ]:
credit_data = pd.read_csv('../input/Credit.csv')
credit_data.info()

In [ ]:
credit_data.head()

In [ ]:
# lets do some graphs to show colinearity and explain its dangers
# TODO finish this off
# need to calculate the rss of regression model using (age and limit) and (rating and limit)


Collinearity refers to the case when two or more features are linked to one another.


In [ ]:
# TODO calculate VIF

# TODO calculate the t and f statistics by hand to better understand which variance to u